# Plotting and Visualization
---
DAT 512 Canisuis College <br>
Professor Paul Lambson<br>
<br>
### Learning Objectives
- Understand theory behind group by
- learn the group by object
- learn how to aggregate
- become familiar with aggregation methods
- Pivots and cross-tabulations
<br>


### Sections
- [How to Think About Group Operations](#how_to_think_about_group_operations)
- [Data Aggregation](#data_aggregations)
- [Apply: General split-apply-combine](#apply)
- [Group Transforms and "Unwrapped" GroupBys](group_transforms_and_unwrapped_groupbys)
- [Pivot Tables and Cross-Tabulation](#pivot_tables_and_cross_tabulations)

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

# How to Think About Group Operations
<a id='how_to_think_about_group_operations'></a>
In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows `(axis="index")` or its columns `(axis="columns")`. Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data.
![split-apply-combine](image/split-apply-combine_.png)

In [ ]:
# create an example dataframe 
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

In [ ]:
# use groupby method to create a groupby object, not calculations have beeb done yet
grouped = df["data1"].groupby(df["key1"])
grouped

In [ ]:
# calling an aggregation method processes the data
grouped.mean()


In [ ]:
# a list of keys can be passesd as a group key, creating a multiindex
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

In [ ]:
# pivot to a dataframe
means.unstack()

In [ ]:
# map new array of correct length
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

In [ ]:
# Frequently, the grouping information is found in the same DataFrame as the data you want to work on. 
# In that case, you can pass column names (whether those are strings, numbers, or other Python objects) 
# as the group keys
df.groupby("key1").mean()

In [ ]:
df.groupby("key2").mean()

In [ ]:
df.groupby(["key1", "key2"]).mean()

In [ ]:
# size produces a count like result showing group sizes
df.groupby(["key1", "key2"]).size()

In [ ]:
# NaNs are not counted by degault
df.groupby("key1", dropna=False).size()
df.groupby(["key1", "key2"], dropna=False).size()

In [ ]:
# A group function similar in spirit to size is count, 
# which computes the number of nonnull values in each group
df.groupby("key1").count()

### Iterating over Groups

In [ ]:
# The object returned by groupby supports iteration
for name, group in df.groupby("key1"):
    print(name)
    print(group)

In [ ]:
# In the case of multiple keys, the first element in the tuple will be a tuple of key values
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)
#! blockend

In [ ]:
# make a dictionary or grouped data frames, if you want to
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

In [ ]:
# mix grouping axis
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

for group_key, group_values in grouped:
    print(group_key)
    print(group_values) 

# Selecting a Column or Subset of Columns

In [ ]:
# after the group by object is created, a column or list of columnsc can be selected
df.groupby(["key1", "key2"])[["data2"]].mean()

In [ ]:
# if a single column is passed as then the result groupby columns is a Series
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped
s_grouped.mean()

# Grouping with Dictionaries and Series

In [ ]:
#! ipython id=78aee68b5f504ff89e6698bd9bbec2b2
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

In [ ]:
#a group correspondence for the columns and want to sum the columns by group
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [ ]:
# the mapper creates an array used for grouping
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

In [ ]:
# same process possible with a Series
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis="columns").count()

# Grouping with Functions

In [ ]:
# a new array is created by evaluating the length of each row index
people.groupby(len).sum()

In [ ]:
# can be combined with multiindex
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

# Grouping with Index Levels

In [ ]:
# example dataframe for multiindex on columns
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

In [ ]:
# pass level by number or name
hier_df.groupby(level="cty", axis="columns").count()

# Data Aggregation
<a id='data_aggregations'></a>
*Aggregations* refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including `mean`, `count`, `min`, and `sum`. You may wonder what is going on when you invoke `mean()` on a GroupBy object. Many common aggregations, such as those found in this table, have optimized implementations. However, you are not limited to only this set of methods.

Funciton name | Description
:--- | :---
`any, all` | Return `True` if any (one or more values) or all non-NA values are “truthy”
`count` | Number of non-NA values
`cummin, cummax` | Cumulative minimum and maximum of non-NA values
`cumsum` | Cumulative sum of non-NA values
`cumprod` | Cumulative product of non-NA values
`first, last` | First and last non-NA values
`mean` | Mean of non-NA values
`median` | Arithmetic median of non-NA values
`min, max` | Minimum and maximum of non-NA values
`nth` | Retrieve value that would appear at position n with the data in sorted order
`ohlc` | Compute four “open-high-low-close” statistics for time series-like data
`prod` | Product of non-NA values
`quantile` | Compute sample quantile
`rank` | Ordinal ranks of non-NA values, like calling Series.rank
`size` | Compute group sizes, returning result as a Series
`sum` |  Sum of non-NA values
`std, var`| Sample standard deviation and variance

In [ ]:
#create a groupby object and pass an argument needed for nsmallest
df
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

In [ ]:
# UDFs can be passed, will perform more slowly
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

In [ ]:
# love to see descriptive statistics
grouped.describe()

# Column-Wise and Multiple Function Application

In [ ]:
# pull in tips dataset
tips = pd.read_csv("examples/tips.csv")
tips.head()

In [ ]:
# create a tips percentage
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

In [ ]:
# create a groupby object with columns, rather than index 
grouped = tips.groupby(["day", "smoker"])

In [ ]:
# pass a single string function name, get a Series
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

In [ ]:
# pass a list of aggregators, get a DataFrame
grouped_pct.agg(["mean", "std", peak_to_peak])

In [ ]:
# pass tuples to rename with custom names rather than function names
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

In [ ]:
# pass a list of functions to apply to all columns
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

In [ ]:
#! ipython id=48aeddf0d8614a9daa16851bf8292777
result["tip_pct"]

In [ ]:
# tuples that indicate names can be passed
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

In [ ]:
# a dictionary can be used to specify aggregation by column
grouped.agg({"tip" : np.max, "size" : "sum"})

In [ ]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

# Returning Aggregated Data Without Row Indexes

In [ ]:
# pass an agrument to supress groupby column to be the new idnex
tips.groupby(["day", "smoker"], as_index=False).mean()

# Apply: General split-apply-combine
<a id='apply'></a>

In [ ]:
# create a function that works with a dataframe
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

In [ ]:
# apply the function to each group
tips.groupby("smoker").apply(top)

In [ ]:
# now with keyword agruments
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

In [ ]:
# can apply group by easily
result = tips.groupby("smoker")["tip_pct"].describe()
result
result.unstack("smoker")

# Supressing the Group Keys

In [ ]:
# surpess by keyword
tips.groupby("smoker", group_keys=False).apply(top)

# Quantile and Bucket Analysis

In [ ]:
#! ipython id=b9e64e9337f247c0a408ea0c7a9961c5
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

In [ ]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

In [ ]:
# establish a custom stats function
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
# apply it to the groupby object
grouped.apply(get_stats)

In [ ]:
# but look how easy it could be
grouped.agg(["min", "max", "count", "mean"])

In [ ]:
# now for similar population sized buckets
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

In [ ]:
# look at counts, evenly sized group, varying on ranges
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

# Example: Filling Missing Values with Group-Specific Values

In [ ]:
#! ipython id=a5f76377f0184195828c907897eb39d1
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

In [ ]:
s.fillna(s.mean())

In [ ]:
#! ipython id=b4f45efcbdd0491f906ca6a4847bab1e
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

In [ ]:
#! ipython id=72f2656d298f413f8720b68b62b51674
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data


In [ ]:
data.groupby(group_key).size()

In [ ]:
data.groupby(group_key).count()

In [ ]:
data.groupby(group_key).mean()

In [ ]:
# create UDF 
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

In [ ]:
# predefined fill values
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

# Example: Random Sampling and Permutation

In [ ]:
# make a deck of cards
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [ ]:
#! ipython id=d54e45a5b95f4e84ab900beaeffc032e
deck.head(13)

In [ ]:
# drawing a 5 card hand
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

In [ ]:
# suppose you wanted 2 cards from each suit
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
# now dropping keys
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

# Example: Group Weighted Average and Correlation

In [ ]:
#
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

In [ ]:
# The weighted average by category would then be
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

In [ ]:
# pull in stock data
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()
close_px.tail(4)

In [ ]:
# pairwise correlation with a column
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [ ]:
# compute percent change on close_px using pct_change
rets = close_px.pct_change().dropna()

In [ ]:
# make a year to group by then 
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

In [ ]:
# inter column correlation
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

# Example: Group-Wise Linear Regression

In [ ]:
# bring in a linear model
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
# find coefficients per group
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

# Group Transforms and “Unwrapped” GroupBys

In [ ]:
# example dataframe
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

In [ ]:
# group means by key:
g = df.groupby('key')['value']
g.mean()

In [ ]:
# can pass a function that computes the mean of a single group to transform
def get_mean(group):
    return group.mean()
g.transform(get_mean)

In [ ]:
# also call by function name as a string
g.transform('mean')

In [ ]:
# manual calculation by group
def times_two(group):
    return group * 2
g.transform(times_two)

In [ ]:
# compute the ranks in descending order for each group
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

In [ ]:
# simple normalize
def normalize(x):
    return (x - x.mean()) / x.std()

In [ ]:
#! ipython id=d06c11e725f744abb0b0309eb2c04526
g.transform(normalize)


In [ ]:
g.apply(normalize)

In [ ]:
#! ipython id=36f1cd239f9a4b218ea0d3dc86776890
g.transform('mean')

In [ ]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

# Pivot Tables and Cross-Tabulation
<a id='group_transforms_and_unwrapped_groupbys'></a>

In [ ]:
# pull in tips dataset
tips = pd.read_csv("examples/tips.csv")
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

In [ ]:
tips.pivot_table(index=["day", "smoker"])

In [ ]:
#  could augment this table to include partial totals by passing margins=True
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

In [ ]:
# could augment this table to include partial totals by passing margins=True
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

In [ ]:
# could pass an agg function
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

In [ ]:
# could fill na values
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

# Cross-Tabulations: Crosstab
<a id='pivot_tables_and_cross_tabulations'></a>

In [ ]:
# bring in some data
from io import StringIO

data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

data = pd.read_table(StringIO(data), sep="\s+")

In [ ]:
data

In [ ]:
# name the indexes
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

In [ ]:
# list of indexes for hierarhcy
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)